In [41]:
import os
import sys
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import lightgbm as lgbm

In [35]:
sys.path.append("../")
from src.utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique

In [3]:
INPUT_DIR = "../input"

In [4]:
book_train = pd.read_parquet(os.path.join(INPUT_DIR, "book_train.parquet/stock_id=15"))
book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


 # 特徴量生成する関数たち

In [5]:
def preprocessor_book(file_path):
    """
    bookデータの特徴量を生成

    CHECK
    ------
    last_secondsのlistに数字を加えれば、最後から何秒みたいなところだけで特徴作れる
    このリストは外で定義したほうがよいかも（preprocessor_tradeでも使う）
    create_feature_dict も外で定義したい
    """
    df = pd.read_parquet(file_path)
    # calculate return etc
    df["wap"] = calc_wap(df)
    df["log_return"] = df.groupby("time_id")["wap"].apply(log_return)

    df["wap2"] = calc_wap2(df)
    df["log_return2"] = df.groupby("time_id")["wap2"].apply(log_return)

    df["wap_balance"] = abs(df["wap"] - df["wap2"])

    df["price_spread"] = (df["ask_price1"] - df["bid_price1"]) / (
        (df["ask_price1"] + df["bid_price1"]) / 2
    )
    df["bid_spread"] = df["bid_price1"] - df["bid_price2"]
    df["ask_spread"] = df["ask_price1"] - df["ask_price2"]
    df["total_volume"] = (df["ask_size1"] + df["ask_size2"]) + (
        df["bid_size1"] + df["bid_size2"]
    )
    df["volume_imbalance"] = abs(
        (df["ask_size1"] + df["ask_size2"]) - (df["bid_size1"] + df["bid_size2"])
    )

    # dict for aggregate
    create_feature_dict = {
        "log_return": [realized_volatility],
        "log_return2": [realized_volatility],
        "wap_balance": [np.mean],
        "price_spread": [np.mean],
        "bid_spread": [np.mean],
        "ask_spread": [np.mean],
        "volume_imbalance": [np.mean],
        "total_volume": [np.mean],
        "wap": [np.mean],
    }

    #####groupby / all seconds
    df_feature = pd.DataFrame(
        df.groupby(["time_id"]).agg(create_feature_dict)
    ).reset_index()

    df_feature.columns = [
        "_".join(col) for col in df_feature.columns
    ]  # time_id is changed to time_id_

    ######groupby / last XX seconds
    last_seconds = [300]

    for second in last_seconds:
        second = 600 - second

        df_feature_sec = pd.DataFrame(
            df.query(f"seconds_in_bucket >= {second}")
            .groupby(["time_id"])
            .agg(create_feature_dict)
        ).reset_index()

        df_feature_sec.columns = [
            "_".join(col) for col in df_feature_sec.columns
        ]  # time_id is changed to time_id_

        df_feature_sec = df_feature_sec.add_suffix("_" + str(second))

        df_feature = pd.merge(
            df_feature,
            df_feature_sec,
            how="left",
            left_on="time_id_",
            right_on=f"time_id__{second}",
        )
        df_feature = df_feature.drop([f"time_id__{second}"], axis=1)

    # create row_id
    stock_id = file_path.split("=")[1]
    df_feature["row_id"] = df_feature["time_id_"].apply(lambda x: f"{stock_id}-{x}")
    df_feature = df_feature.drop(["time_id_"], axis=1)

    return df_feature

In [6]:
def preprocessor_trade(file_path):
    """
    tradeデータの特徴量を生成

    CHECK
    ------
    last_secondsのlistに数字を加えれば、最後から何秒みたいなところだけで特徴作れる
    このリストは外で定義したほうがよいかも（preprocessor_tradeでも使う）
    aggregate_dictionary も外で定義したい
    """
    df = pd.read_parquet(file_path)
    df["log_return"] = df.groupby("time_id")["price"].apply(log_return)

    aggregate_dictionary = {
        "log_return": [realized_volatility],
        "seconds_in_bucket": [count_unique],
        "size": [np.sum],
        "order_count": [np.mean],
    }

    df_feature = df.groupby("time_id").agg(aggregate_dictionary)

    df_feature = df_feature.reset_index()
    df_feature.columns = ["_".join(col) for col in df_feature.columns]

    ######groupby / last XX seconds
    last_seconds = [300]

    for second in last_seconds:
        second = 600 - second

        df_feature_sec = (
            df.query(f"seconds_in_bucket >= {second}")
            .groupby("time_id")
            .agg(aggregate_dictionary)
        )
        df_feature_sec = df_feature_sec.reset_index()

        df_feature_sec.columns = ["_".join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix("_" + str(second))

        df_feature = pd.merge(
            df_feature,
            df_feature_sec,
            how="left",
            left_on="time_id_",
            right_on=f"time_id__{second}",
        )
        df_feature = df_feature.drop([f"time_id__{second}"], axis=1)

    df_feature = df_feature.add_prefix("trade_")
    stock_id = file_path.split("=")[1]
    df_feature["row_id"] = df_feature["trade_time_id_"].apply(
        lambda x: f"{stock_id}-{x}"
    )
    df_feature = df_feature.drop(["trade_time_id_"], axis=1)

    return df_feature

In [7]:
def preprocessor(data_dir, list_stock_ids, is_train=True):
    from joblib import Parallel, delayed  # parallel computing to save time

    df = pd.DataFrame()

    def for_joblib(stock_id):
        if is_train:
            file_path_book = os.path.join(
                data_dir, f"book_train.parquet/stock_id={stock_id}"
            )
            file_path_trade = os.path.join(
                data_dir, f"trade_train.parquet/stock_id={stock_id}"
            )
        else:
            file_path_book = os.path.join(
                data_dir, f"book_test.parquet/stock_id={stock_id}"
            )
            file_path_trade = os.path.join(
                data_dir, f"trade_test.parquet/stock_id={stock_id}"
            )

        df_tmp = pd.merge(
            preprocessor_book(file_path_book),
            preprocessor_trade(file_path_trade),
            on="row_id",
            how="left",
        )
        return pd.concat([df, df_tmp])

    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
    )

    df = pd.concat(df, ignore_index=True)
    return df

In [33]:
def calc_model_importance(model, feature_names=None, importance_type="gain"):
    importance_df = pd.DataFrame(
        model.feature_importance(importance_type=importance_type),
        index=feature_names,
        columns=["importance"],
    ).sort_values("importance")
    return importance_df


def plot_importance(importance_df, title="", save_filepath=None, figsize=(8, 12)):
    fig, ax = plt.subplots(figsize=figsize)
    importance_df.plot.barh(ax=ax)
    if title:
        plt.title(title)
    plt.tight_layout()
    if save_filepath is None:
        plt.show()
    else:
        plt.savefig(save_filepath)
    plt.close()
    
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

# run

In [8]:
%%time
file_path = os.path.join(INPUT_DIR, "book_train.parquet/stock_id=0")
df = preprocessor_book(file_path)
df.columns

CPU times: user 4.05 s, sys: 38.8 ms, total: 4.09 s
Wall time: 4.09 s


Index(['log_return_realized_volatility', 'log_return2_realized_volatility',
       'wap_balance_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'volume_imbalance_mean', 'total_volume_mean',
       'wap_mean', 'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'row_id'],
      dtype='object')

In [9]:
%%time

file_path = os.path.join(INPUT_DIR, "trade_train.parquet/stock_id=0")
preprocessor_trade(file_path)

CPU times: user 2 s, sys: 21 ms, total: 2.02 s
Wall time: 2.01 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,row_id
0,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429,0-5
1,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000,0-11
2,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667,0-16
3,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111,0-31
4,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091,0-62
...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.001162,35.0,2365.0,3.257143,0-32751
3826,0.001411,28,4547,3.892857,0.001066,12.0,2161.0,4.250000,0-32753
3827,0.001521,36,4250,3.500000,0.001242,22.0,2294.0,3.727273,0-32758
3828,0.001794,53,3217,2.150943,0.001404,25.0,1627.0,1.920000,0-32763


In [10]:
list_stock_ids = [0, 1]
tmp = preprocessor(INPUT_DIR, list_stock_ids, is_train=True)
# stock_idの番号はrow_idに付与されてる（ハイフンの←側の数字）

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    6.5s finished


In [11]:
tmp.row_id

0           0-5
1          0-11
2          0-16
3          0-31
4          0-62
         ...   
7655    1-32751
7656    1-32753
7657    1-32758
7658    1-32763
7659    1-32767
Name: row_id, Length: 7660, dtype: object

# training set

In [19]:
train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
train_ids = train.stock_id.unique()
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [20]:
%%time
df_train = preprocessor(INPUT_DIR, list_stock_ids=train_ids, is_train=True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   19.9s


CPU times: user 756 ms, sys: 40.3 ms, total: 796 ms
Wall time: 1min 16s


[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  1.3min finished


In [21]:
train["row_id"] = train["stock_id"].astype(str) + "-" + train["time_id"].astype(str)
train = train[["row_id", "target"]]
df_train = train.merge(df_train, on=["row_id"], how="left")
df_train.head()

,row_id,target,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,...,total_volume_mean_300,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0-5,0.004136,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,...,294.928058,1.003753,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429
1,0-11,0.001445,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,...,484.521739,1.000397,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000
2,0-16,0.002168,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,...,455.235294,0.998685,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667
3,0-31,0.002195,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,...,418.169811,0.998436,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111
4,0-62,0.001747,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,...,407.584270,0.999488,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091


# test set

In [22]:
test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))
test_ids = test.stock_id.unique()

In [23]:
%%time
df_test = preprocessor(INPUT_DIR, list_stock_ids=test_ids, is_train=False)
df_test = test.merge(df_test, on=["row_id"], how="left")

CPU times: user 8.77 ms, sys: 601 µs, total: 9.37 ms
Wall time: 70.6 ms


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [24]:
df_test

,stock_id,time_id,row_id,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,...,total_volume_mean_300,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0,4,0-4,0.000294,0.000252,0.000145,0.000557,0.000393,-0.000115,164.666667,...,NaN,NaN,0.000295,3.0,201.0,3.666667,NaN,NaN,NaN,NaN
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# target encoding by stock_id

In [26]:
# stock_id target encoding
df_train["stock_id"] = df_train["row_id"].apply(lambda x: x.split("-")[0])
df_test["stock_id"] = df_test["row_id"].apply(lambda x: x.split("-")[0])

stock_id_target_mean = df_train.groupby("stock_id")["target"].mean()
df_test["stock_id_target_enc"] = df_test["stock_id"].map(
    stock_id_target_mean
)  # test_set

# training
#### CHECK
# この辺、あんまり分かってない
# oofでtarget encordingしてるんだと思うけども
# 自分で書き直したい
tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits=10, shuffle=True, random_state=19911109)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby("stock_id")["target"].mean()

    tmp[idx_2] = df_train["stock_id"].iloc[idx_2].map(target_mean)
df_train["stock_id_target_enc"] = tmp

# model

In [28]:
DO_FEAT_IMP = False
if len(df_test) == 3:
    DO_FEAT_IMP = True

In [32]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [39]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [36]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

In [37]:
kf = KFold(n_splits=5, random_state=19901028, shuffle=True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0                         # validation score

gain_importance_list = []
split_importance_list = []

In [40]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold :", fold+1)
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    
    #RMSPE weight
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train,y_train,weight = weights)

    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid,y_valid,reference = lgbm_train,weight = weights)
    
    # model 
    model = lgbm.train(params=params,
                      train_set=lgbm_train,
                      valid_sets=[lgbm_train, lgbm_valid],
                      num_boost_round=5000,         
                      feval=feval_RMSPE,
                      verbose_eval=100,
                      categorical_feature = ['stock_id']                
                     )
    
    # validation 
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)
    
    # --- calc model feature importance ---
    if DO_FEAT_IMP:    
        feature_names = X_train.columns.values.tolist()
        gain_importance_df = calc_model_importance(
            model, feature_names=feature_names, importance_type='gain')
        gain_importance_list.append(gain_importance_df)

        split_importance_df = calc_model_importance(
            model, feature_names=feature_names, importance_type='split')
        split_importance_list.append(split_importance_df)

Fold : 1


NameError: name 'lgbm' is not defined

In [ ]:
def calc_mean_importance(importance_df_list):
    mean_importance = np.mean(
        np.array([df['importance'].values for df in importance_df_list]), axis=0)
    mean_df = importance_df_list[0].copy()
    mean_df['importance'] = mean_importance
    return mean_df

In [ ]:
if DO_FEAT_IMP:
    mean_gain_df = calc_mean_importance(gain_importance_list)
    plot_importance(mean_gain_df, title='Model feature importance by gain')
    mean_gain_df = mean_gain_df.reset_index().rename(columns={'index': 'feature_names'})
    mean_gain_df.to_csv('gain_importance_mean.csv', index=False)

In [ ]:
y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis = 1)

In [ ]:
target = np.zeros(len(X_test))

#light gbm models
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)

In [ ]:
y_pred = y_pred.assign(target = target)

In [ ]:
y_pred.to_csv('submission.csv',index = False)